[View in Colaboratory](https://colab.research.google.com/github/ShriPunta/Netflix-Graph-Dataset-Project/blob/master/ParsingTheCombinedFile.ipynb)

In [0]:
!pip install -U -q PyDrive


In [0]:
#!pip install python-igraph

**Setup all the Authentication for PyDrive**

In [0]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files

In [0]:
%matplotlib inline

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


---

**Method to combine all the records**

In [0]:
def merge_all_texts(file_list):
  import shutil

  with open('AllMerged.txt','wb') as wfd:
    for f in [file_list]:
      with open(f,'rb') as fd:
        shutil.copyfileobj(fd, wfd, 1024*1024*10)
        #10MB per writing chunk to avoid reading big file into memory.


** *Method*: To read the contents of a local Google File**



In [0]:
def read_drive_file_into_variable(file_id_to_read):
  drive_service = build('drive', 'v3')
  request = drive_service.files().get_media(fileId=file_id_to_read)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    status, done = downloader.next_chunk()
    prog = int(status.progress() * 100)
    print("Download "+str(prog))
  
  #Set the pointer to the start
  downloaded.seek(0)
  #print('Downloaded file contents are: {}'.format(downloaded.read()))
  
  #Read Everthing into a variable called "View", its in a "Bytes" datatype
  view = downloaded.read1(-1)
  #len(view)
  
  #decode Bytes to String format
  decoded = view.decode(encoding="utf-8")
  #type(decoded)
  
  #Convert this continous string to List for each new line
  variable_to_set = decoded.splitlines()
  #type(splitted)
  
  return variable_to_set
  
  

---

**Read the main two files into variables**

In [0]:
movie_Title_file_id = '0B8qgJkz0ynl8czBBNG9qZ2JPeW9RaWVuZktobUE1b29qaER3'
part_rating_file_id = '1iABtudmoCPxcFYiYQ0cQwfcQCZtfHsHd'
test_file_id = '11QhP0HwV7x6huJX3-JqJzGv7shMiZVPH'
merged_file_id = '1zhm2Wo8qBun5p01z_hjZlkww1TIbkTJy'
merge_first3_files = '1R8g2MWa1Czpzt7EAGUfcvADJ-yIB2Yi9'

In [0]:
key_file_list = read_drive_file_into_variable(movie_Title_file_id)

In [0]:
merged_file3_list = read_drive_file_into_variable(merge_first3_files)

In [0]:
#merged_file_list = read_drive_file_into_variable(merged_file_id)

In [0]:
#rating_file_list = read_drive_file_into_variable(part_rating_file_id)

In [0]:
#test_file_list = read_drive_file_into_variable(test_file_id)

In [0]:
final_file_list_to_use = merged_file3_list

---

**Convert the key file into a dataframe**

In [0]:
#Read the key file into a Panda dataframe
key_file_df = pd.DataFrame([sub.split(",") for sub in key_file_list],columns = ["MovieId","Year","Name","Genre1","Genre2","Genre3"])

key_file_df.head()

In [0]:
key_file_df.info(memory_usage = 'deep')

**Clean the dataframe**

In [0]:
#key_file_df['Genre'] = key_file_df[['Genre1','Genre2','Genre3']].apply(lambda x: ''.join(x), axis=1)
# key_file_df.head()

def clean_the_movie_key_df(key_file_df):
  #Convert NaN to blank
  key_file_df = key_file_df.replace(np.nan, '', regex=True)
    
  #Combine the Genres into a single column
  key_file_df['Genre'] = key_file_df['Genre1'] + ',' + key_file_df['Genre2'] + ','+ key_file_df['Genre3']
  
  

  #Clean the column by removing double quotes; Also converts to string
  key_file_df['Genre'] = key_file_df['Genre'].str.replace('"', '')

  #Drop the unnecessary columns
  key_file_df.drop(['Genre1','Genre2','Genre3'], axis=1, inplace=True)
  
  #Convert to numeric
  key_file_df['MovieId'] = pd.to_numeric(key_file_df['MovieId'],errors = 'coerce')
  
  #Convert to numeric
  key_file_df['Year'] = pd.to_numeric(key_file_df['Year'],errors = 'coerce')
  
  #Convert to String
  key_file_df['Name'] = key_file_df['Name'].astype('str')
  
  
  #Set MovieId as the Index
  key_file_df.set_index('MovieId',inplace=True)
  
  #Drop any rows which NaN or NULL
  key_file_df.dropna(axis=0, how='any',inplace=True)

  return key_file_df


In [0]:
key_file_df = clean_the_movie_key_df(key_file_df)

In [0]:
#Gives detailing on how many null or na there are
key_file_df.isnull().sum()

In [0]:
key_file_df.info(memory_usage='deep')

** We store the index(which are the movie Ids) as a list; to be used as a reference later**

In [0]:
key_movie_values = key_file_df.index.values
key_movie_values


---

**Create a Map of each movie with the number of ratings it has**

In [0]:
def fill_dict_count(list_to_iterate):
  each_movie_votes = dict()
  for ele in list_to_iterate:
    if ele.find(':') != -1:
      key_to_search = (ele.split(':'))[0]
      
      #Get a map of how many ratings are there per movie
      each_movie_votes[key_to_search] = 0
    else:
      each_movie_votes[key_to_search] +=1
   
  return each_movie_votes
      
    

**---> PARSNG and SAMPLING <---**

**SAMPLING  : It takes the 23 million ratings, and only keeps limited number of ratings per movie (This is controlled by the 'limiter' variable).**

**We only want values whose genre we have, hence we weed out those movies, who are not present in the movie_Title_file**

**----**
  
**PARSING :  It also adds a comma separated value of the movie id to the tuple. (It is split later to create a dataframe).**

In [0]:
def do_sampling(list_to_iterate,file_rating_dict):
  flag=True
  percent = 0.5
  count = 0
  refined_list = []
  key_to_search = 0
  limiter = 1
    
  for ele in list_to_iterate:
    
    #If the element has a ':' then its a movie id
    if ele.find(':') != -1:
      count = 1
      #we will remove the ':' from the tuple
      #This is done as we can then successfully apply the pd.to_numeric method to drastically reduce the dataframe size
      key_to_search = (ele.split(':'))[0]
      
      #We want only those movies which are present in the movie_Title_file
      if int(key_to_search) in key_movie_values:
        #If this movie is present, set flag as true
        flag = True
        
        #Set Limiter value
        limiter = round(file_rating_dict.get(key_to_search) * percent)
      else:
        flag = False
    else:
      #If the flag is false(i.e. movie not found) or if the count more than the limit; SKIP the record
      if count > limiter or flag is False:
        continue
      #Add a comma separated value of the movieId to split and form a column later
      refined_list.append(ele+','+ str(key_to_search))
      count+=1
      
  return refined_list

In [0]:
count_strength = fill_dict_count(final_file_list_to_use)

In [0]:
refined_list = do_sampling(final_file_list_to_use,count_strength)


In [0]:
max_values = sorted(set(count_strength.values()),reverse=True)
print(max_values)

In [0]:
print(len(count_strength.keys()))

---
**Get a judgement of the size of the earlier list and size after sampling**


In [0]:
from sys import getsizeof
#print("Total number of elements -->",len(rating_file_list),"  Size they occupy in bytes -->",getsizeof(rating_file_list))

print("Total number of Elements in rating_file_list list -->",len(final_file_list_to_use),"  Size they occupy in bytes -->",getsizeof(final_file_list_to_use))
print("Total number of Elements in refined_list list -->",len(refined_list),"  Size they occupy in bytes -->",getsizeof(refined_list))

---

**Convert rating File into dataframe**

In [0]:
#Read the key file into a Panda dataframe
test_file_df = pd.DataFrame([sub.split(",") for sub in refined_list],columns = ["UserId","Rating","YearWatched","MovieId"])

#Initialize a blank column called MovieId ; to be filled latter
test_file_df[['MovieId']] = test_file_df[['MovieId']].apply(pd.to_numeric)

#Need memory optimization, convert the object type to numeric
test_file_df[['UserId']] = test_file_df[['UserId']].apply(pd.to_numeric)

#Need memory optimization, convert the object type to numeric
test_file_df[['Rating']] = test_file_df[['Rating']].apply(pd.to_numeric)

#Convert the column to datetime and keep only the year
test_file_df['YearWatched'] = pd.to_datetime(test_file_df['YearWatched']).dt.year

#Need memory optimization, convert the object type to numeric
test_file_df['UserId'] = pd.to_numeric(test_file_df['UserId'],errors='ignore')

In [0]:
#Applying these changes in datatypes brought memory usage from 5.4 GB to 2 GB
test_file_df.info(memory_usage = 'deep')

In [0]:
#Drop any rows which NaN or NULL
test_file_df.dropna(axis=0, how='any',inplace=True,thresh = 2)


In [0]:
test_file_df.head(2)

In [0]:
#Create a matrix which is the same size of test_file_df matrix, but instead has True/False about which if the value is NaN
the_NaN_matrix = test_file_df.isnull().sum()
the_NaN_matrix

In [0]:
test_file_df['Genre'] = ''


#Copy the Genre Column from key file pandas dataframe
test_file_df['Genre'] = test_file_df['MovieId'].map(key_file_df['Genre'])

In [0]:
#To download the Panda Data Frame
#test_file_df.to_csv('test_file_df.csv',index=False)
#files.download('test_file_df.csv')



---



---
 ** *Method*: To see the GPU usage**

In [0]:
def checkGPU():
  # memory footprint support libraries/code
  !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
  !pip install gputil
  !pip install psutil
  !pip install humanize
  import psutil
  import humanize
  import os
  import GPUtil as GPU
  GPUs = GPU.getGPUs()
  # XXX: only one GPU on Colab and isn’t guaranteed
  gpu = GPUs[0]
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  


---

In [0]:
checkGPU()



---

## **CODE GRAVEYARD**



---



In [0]:
#This Method gives the Numpy shape error; reason unidentified

# def do_sampling(list_to_iterate):
#   flag=True
#   #Limiter variable
#   limiter = 100
#   count = 0
#   #variable to be returned
#   refined_list = []
#   key_to_search = 0
#   movieId_to_search=''
#   for ele in list_to_iterate:
#     #If the element has a ':' then its a movie id
#     if ele.find(':') != -1:
#       count = 1
#       #we will remove the ':' from the tuple
#       #This is done as we can then successfully apply the pd.to_numeric method to drastically reduce the dataframe size
#       movieId_to_search = ele.replace(':','')
#       #We want only those movies which are present in the movie_Title_file
      
#       if movieId_to_search in key_movie_values:
#         print('found')
#         #If this movie is present, set flag as true
#         flag = True
#         refined_list.append(movieId_to_search)
#       else:
#         flag = False
#     else:
#       #If the flag is false(i.e. movie not found) or if the count more than the limit; SKIP the record
#       if count > limiter or flag is False:
#         continue
#       #Add a comma separated value of the movieId to split and form a column later
#       ele += ',' +str(movieId_to_search)
      
#       refined_list.append(ele)
      
#       count+=1
#   return refined_list



---

**BELOW 2 blocks have been deemed deprecated **



---



In [0]:
#Ultra fast method to get the indexes of the rows we need to drop; Reduces time from a minute to a second
#For records which only had the movie id and nothing else, will have NaN for columns other than the first column
#fancy_list = test_file_df[the_NaN_matrix['Rating'] == True].index.values

In [0]:
#type(fancy_list[2])

---
**This below block is deprecated, due to the faster and optimized ways.**

In [0]:
# #We will store the rows to drop in this
# rows_to_drop = []
# currentMovieId = 0
# list_movieId = []


# #Iterate over the dataframe to split it
# for index, row in test_file_df.iterrows():
#   if the_NaN_matrix.iloc[index,2]:
#     #If entered it means that there is a colon on the row and it is a movie id
#     #row gives the first character on that row
#     #currentMovieId = row['UserId']
#     rows_to_drop.append(index)
#   else:
#     list_movieId.append(currentMovieId)
#     #print("Not")
#     #Assign the movie id to the consecutive rows
#     #test_file_df.iloc[index,3] = currentMovieId

# #Create a series object from the list
# #MovieIdSeries = pd.Series(list_movieId)  



---



In [0]:
#Drop the rows which had the movie Id and the columns to get a seamless dataframe
#test_file_df.drop(test_file_df.index[fancy_list],inplace=True)




---





---

